__STATISTICAL APPLICATION__

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
from scipy.stats import ttest_ind_from_stats
from scipy.stats import ttest_ind
from scipy.stats import t
from scipy.stats import norm
from numpy.random import seed
import math

In [2]:
#load the original dataset and assign it to dfmallcustomer dataframe
dfmallcustomer= pd.read_csv("datasets/Mall_Customers.csv")

In [3]:
dfmallcustomer.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


__STUDENT'S T-TEST__

H0 = There is no significant difference in spending scores between males and females.
H1 = There is a significant difference in speding scores betwewen males and females.

In [4]:
#calculate the number of male users
male = dfmallcustomer[dfmallcustomer.Gender == 'Male']
male.shape

(88, 5)

In [5]:
#calculate the number of fewmale users
female = dfmallcustomer[dfmallcustomer.Gender == 'Female']
female.shape

(112, 5)

In [6]:
#calculate the mean and standard deviations of males
male_ss_mean = male['Spending Score (1-100)'].mean()
print(male_ss_mean)

male_ss_std = male['Spending Score (1-100)'].std()
print(male_ss_std)

48.51136363636363
27.896769605833597


In [7]:
#calculate the mean and standard deviations of females
female_ss_mean = female['Spending Score (1-100)'].mean()
print(female_ss_mean)

female_ss_std = female['Spending Score (1-100)'].std()
print(female_ss_std)

51.526785714285715
24.114949877478647


In [8]:
#difference between the mean of the spending scores of males and females
mean_difference= female_ss_mean - male_ss_mean
print("The difference between the means of the spending scores of males and females is ", mean_difference)

The difference between the means of the spending scores of males and females is  3.015422077922082


In [9]:
# calculate the standard error

SE=((female_ss_std**2)/372311 + (male_ss_std**2)/476439)**0.5
print("Standard error:", SE)

Standard error: 0.056527683778373194


In [11]:
#manual calculation of t and p values
male_ss = dfmallcustomer['Spending Score (1-100)'][dfmallcustomer.Gender == 'Male']  #assign male spendeing scores
female_ss = dfmallcustomer['Spending Score (1-100)'][dfmallcustomer.Gender == 'Female'] #assign female spending scores
mean_male = np.mean(male_ss) #mean of male spending score
print('Mean Male:', mean_male)
mean_female = np.mean(female_ss) #mean of female spending score
print('Mean Female:', mean_female)
std_dev_male = np.std(male_ss) #standard deviation of male spending scores
print('Standard Deviation Male:', std_dev_male)
std_dev_female = np.std(female_ss) #standard deviation of female spending scores
print('Standard Deviation Female:', std_dev_female)

Mean Male: 48.51136363636363
Mean Female: 51.526785714285715
Standard Deviation Male: 27.737812359739767
Standard Deviation Female: 24.007052468801657


In [12]:
#calculate t value manually
n0 = len(male_ss)
n1= len(female_ss)
std0 = male_ss.std()
std1= female_ss.std()
mean0 = mean_male
mean1= mean_female
sp = np.sqrt( ((n0-1)*(std0)**2 + (n1-1)*(std1)**2)/ (n0+n1-2) )
t_ = (mean1 - mean0)/(sp * np.sqrt(1/n0 + 1/n1)) 
print(t_)

0.8190464150660334


In [13]:
# Use 0.05 Significance level in two sample t-test
t_val=((male_ss_mean - female_ss_mean)-0)/SE
print(t_val)

-53.34416477675928


In [14]:
#calculate p value manually 
p_value = (1 - t(n0 + n1 - 1).cdf(t_)) * 2
p_value

0.4137397159674374

In [15]:
#calculate t and p values using scipy
ttest_ind(male_ss, female_ss)

Ttest_indResult(statistic=-0.8190464150660333, pvalue=0.4137446589852176)

__INTERPRETATION__: Because the p-value is higher than 0.05, we reject null hypothesis and say that there is significant difference between the means of males and females ages. This aligns with what I found in datastorytelling.

__BOOTSTRAP METHOD and CONFIDENCE INTERVAL__

In [16]:
from numpy.random import seed

In [19]:
np.random.seed(47)
N_rep = 10000

# set up an empty array
bs_replicates = np.empty(N_rep)

# getting 10000 bootstrap replicates (10000) 
# replicates indicate the statistic value

for i in range(N_rep):
    bs_sample = np.random.choice(dfmallcustomer['Spending Score (1-100)'], size=len(dfmallcustomer['Spending Score (1-100)']))
    bs_replicates[i] = bs_sample.mean()

In [20]:
# estimating the same 95% confidence interval lower limit 
# since it's the lower limit, set the area under the curve as 5%
lower_limit = np.percentile(bs_replicates, [5])[0]

print("The lower limit of the 95% confidence interval is: {}".format(lower_limit))

The lower limit of the 95% confidence interval is: 47.215


__INTERPRETATION:__ When I performed 10000 replicates immediately after setting the random seed to 47, I got the the value 47.215.

In [21]:
# making a function since we will use this logic often
# (Reference: Datacamp / Statistical Thinking in Python (Part 2) / Generating many Bootstrap Replicates)

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates. 
    Func refers to the type of statistic we want (np.mean / np.median etc.)"""
    
    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_sample = np.random.choice(data, len(data))
        bs_replicates[i] = func(bs_sample)  # store the statistic from each bs_sample into the new array

    return bs_replicates

In [23]:
# Male and female groups in arrays
male_bts = np.array(dfmallcustomer[dfmallcustomer['Gender']== "Male"]['Spending Score (1-100)'])
female_bts = np.array(dfmallcustomer[dfmallcustomer['Gender']== "Female"]['Spending Score (1-100)'])

# Difference of the standard deviation of the original sample (observed)
obs_difference_std = np.std(male_bts) - np.std(female_bts)
obs_difference_std

3.7307598909381063

In [24]:
# difference in bootstrap relicates (for N=10000)
bs_diff_std = draw_bs_reps(male_bts, np.std, size=10000) - draw_bs_reps(female_bts, np.std, size=10000)
    
# getting the 95% confidence interval
ci_95 = np.percentile(bs_diff_std,[2.5, 97.5])
ci_95

print("The 95% confidence interval of the difference in standard deviation\
      between two groups is: [{} , {}]".format(ci_95[0],ci_95[1]))

The 95% confidence interval of the difference in standard deviation      between two groups is: [-0.16439370371649292 , 7.500105521493774]



__INTERPRETATION:__ My Null and Alternative Hypothesis were as follows:

$H_0$ : there is no difference in standard deviations between males and females i.e. $\mu_d$ = 0

$H_a$ : there is a difference in standard deviations between males and females i.e. $\mu_d$ $\neq$ 0

My Bootstrap replicates with a 95% confidence interval indicate that the difference in standard deviations between the two groups have a 95% chance of lying within [-0.16439370371649292 , 7.500105521493774]

In [25]:
# # Shifting the Dataset so that the two groups have equal means

# First calculating the combined mean
combined_mean = np.mean(np.concatenate((male_bts, female_bts)))

# Generate the shifted dataset 
male_shifted = male_bts - np.mean(male_bts) + combined_mean
female_shifted =  female_bts - np.mean(female_bts) + combined_mean

In [27]:

# Draw the bootstrap replicates from the shifted dataset
bs_replicates_male = draw_bs_reps(male_shifted, np.mean, size=10000)
bs_replicates_female = draw_bs_reps(female_shifted, np.mean, size=10000)

In [28]:
# Get the differences for the bootstrap simulated sample
bs_differences = bs_replicates_male - bs_replicates_female


# Get the observed difference from the actual dataset
obs_diff = np.mean(male_bts) - np.mean(female_bts)
obs_diff

-3.015422077922082

In [29]:
# Calculate the p-value by comparing the bootstrap replicates against the observed difference of the means
# The fraction of values WITHIN bootstrap replicates array that meet a certain criteria against the obs_diff

p = np.sum(bs_differences >= obs_diff)/ len(bs_differences)
print('p-value =', p)

p-value = 0.7956


__INTERPRETATION:__ Under the null hypothesis, we get a p-value of 0.7956. Thus it is sufficiently likely that the null hypothesis is true and thus I retain the null hyphothesis. There is no significant difference in spending scores between females and males..